# Unit00 Google Colab環境設定教學 | 實作演練

本 Notebook 提供 Google Colab 平台的實際操作演練，包含：

## 目標
- 實際操作 Colab 介面與基本功能
- 練習檔案上傳與 Google Drive 掛載
- 安裝與使用 Python 套件
- 測試 GPU/TPU 運算資源
- 完成一個簡單的化工數據分析範例

**建議**: 請在 Google Colab 環境中開啟此檔案，以便實際操作所有功能。

---
## 1. 環境檢查與基本資訊

In [ ]:
# 檢查 Python 版本與系統資訊
import sys
import platform

print("=== Python Environment Information ===")
print(f"Python Version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Processor: {platform.processor()}")

# 檢查是否在 Colab 環境
try:
    import google.colab
    IN_COLAB = True
    print("\n✓ 目前在 Google Colab 環境中執行")
except ImportError:
    IN_COLAB = False
    print("\n✗ 目前不在 Google Colab 環境中")
    print("  建議在 Colab 中開啟此筆記本以使用完整功能")

In [ ]:
# 查看系統資源
!echo "=== CPU Information ==="
!cat /proc/cpuinfo | grep "model name" | head -1
!echo ""
!echo "=== Memory Information ==="
!cat /proc/meminfo | grep "MemTotal"
!echo ""
!echo "=== Disk Space ==="
!df -h | grep -E "Filesystem|/content"

---
## 2. 檔案系統與 Google Drive 掛載

In [ ]:
# 查看當前目錄結構
import os
from pathlib import Path

print("=== Current Working Directory ===")
print(f"Current Directory: {os.getcwd()}")
print(f"\nDirectory Contents:")
!ls -la /content/

In [ ]:
# 掛載 Google Drive
# 執行此 cell 會出現授權連結，需點選並授權

from google.colab import drive

print("正在掛載 Google Drive...")
drive.mount('/content/drive', force_remount=True)
print("\n✓ Google Drive 掛載成功！")
print(f"掛載路徑: /content/drive/My Drive/")

# 列出 Drive 根目錄內容
print("\nGoogle Drive 根目錄內容:")
!ls "/content/drive/My Drive/" | head -10

---
## 3. 套件管理與安裝

In [ ]:
# 檢查預裝套件版本
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import sklearn
import tensorflow as tf

print("=== 預裝套件版本 ===")
print(f"NumPy:        {np.__version__}")
print(f"Pandas:       {pd.__version__}")
print(f"Matplotlib:   {matplotlib.__version__}")
print(f"Seaborn:      {sns.__version__}")
print(f"Scikit-learn: {sklearn.__version__}")
print(f"TensorFlow:   {tf.__version__}")

In [ ]:
# 安裝額外套件示範
# 這裡安裝一些化工領域可能用到的套件

print("正在安裝額外套件...")

# 靜默安裝（-q 參數減少輸出訊息）
!pip install -q xgboost
!pip install -q lightgbm

print("✓ 套件安裝完成！")

# 驗證安裝
import xgboost as xgb
import lightgbm as lgb

print(f"\nXGBoost version: {xgb.__version__}")
print(f"LightGBM version: {lgb.__version__}")

---
## 4. GPU/TPU 運算資源檢查

In [ ]:
# 檢查 GPU 可用性 (TensorFlow)
import tensorflow as tf

print("=== TensorFlow GPU Information ===")
print(f"TensorFlow Version: {tf.__version__}")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\n✓ GPU 可用！")
    print(f"偵測到 {len(gpus)} 個 GPU:")
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu.name}")
    
    # 設定 GPU 記憶體動態分配
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("\n✓ GPU 記憶體動態分配已啟用")
    except RuntimeError as e:
        print(f"設定失敗: {e}")
else:
    print("\n✗ 未偵測到 GPU")
    print("請至「執行階段」→「變更執行階段類型」→ 選擇「GPU」")

In [ ]:
# 使用 nvidia-smi 查看 GPU 詳細資訊
print("=== GPU Hardware Details (nvidia-smi) ===\n")
!nvidia-smi

---
## 5. 實作範例：反應器數據分析

接下來我們將進行一個完整的化工數據分析流程，展示如何在 Colab 中進行資料處理、視覺化與機器學習。

### 5.1 產生模擬數據

In [ ]:
# 載入必要套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 設定隨機種子
np.random.seed(42)

# 產生模擬的反應器操作數據
n_samples = 200

print("=== 產生模擬反應器操作數據 ===")
print(f"樣本數量: {n_samples}")

# 操作參數
temperature = np.random.normal(350, 20, n_samples)  # Temperature (°C)
pressure = np.random.normal(5, 0.5, n_samples)      # Pressure (bar)
flow_rate = np.random.normal(100, 10, n_samples)    # Flow rate (L/min)

# 產物轉化率（模擬真實關係：受溫度和壓力影響）
conversion = (
    0.3 * (temperature - 300) / 50 + 
    0.4 * (pressure - 4) / 2 + 
    0.1 * (flow_rate - 90) / 20 +
    np.random.normal(0, 0.05, n_samples)
)
conversion = np.clip(conversion, 0, 1)  # 限制在0-1之間

# 建立 DataFrame
data = pd.DataFrame({
    'Temperature': temperature,
    'Pressure': pressure,
    'Flow_Rate': flow_rate,
    'Conversion': conversion
})

print("\n✓ 數據產生完成！")
print(f"\nDataset shape: {data.shape}")
print("\nFirst 5 rows:")
print(data.head())
print("\nBasic statistics:")
print(data.describe())

### 5.2 資料視覺化

In [ ]:
# 繪製資料分布與關係圖
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

fig, axes = plt.subplots(2, 2)

# 1. Temperature Distribution
axes[0, 0].hist(data['Temperature'], bins=30, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Temperature Distribution')
axes[0, 0].set_xlabel('Temperature (°C)')
axes[0, 0].set_ylabel('Frequency')

# 2. Pressure Distribution
axes[0, 1].hist(data['Pressure'], bins=30, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Pressure Distribution')
axes[0, 1].set_xlabel('Pressure (bar)')
axes[0, 1].set_ylabel('Frequency')

# 3. Temperature vs Conversion
axes[1, 0].scatter(data['Temperature'], data['Conversion'], alpha=0.5)
axes[1, 0].set_title('Temperature vs Conversion')
axes[1, 0].set_xlabel('Temperature (°C)')
axes[1, 0].set_ylabel('Conversion')

# 4. Pressure vs Conversion
axes[1, 1].scatter(data['Pressure'], data['Conversion'], alpha=0.5, color='green')
axes[1, 1].set_title('Pressure vs Conversion')
axes[1, 1].set_xlabel('Pressure (bar)')
axes[1, 1].set_ylabel('Conversion')

plt.tight_layout()
plt.show()

print("✓ 視覺化完成")

### 5.3 相關性分析

In [ ]:
# 計算相關係數矩陣
correlation_matrix = data.corr()

print("=== Correlation Matrix ===")
print(correlation_matrix)

# 繪製熱力圖
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap')
plt.show()

print("\n✓ 相關性分析完成")

### 5.4 建立機器學習模型

In [ ]:
# 準備資料
X = data[['Temperature', 'Pressure', 'Flow_Rate']]
y = data['Conversion']

# 分割訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("=== Data Split ===")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

# 建立並訓練線性迴歸模型
print("\n=== Training Linear Regression Model ===")
model = LinearRegression()
model.fit(X_train, y_train)
print("✓ Model trained successfully!")

# 進行預測
y_pred = model.predict(X_test)

# 評估模型
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n=== Model Performance ===")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

# 顯示模型係數
print(f"\n=== Model Coefficients ===")
for feature, coef in zip(X.columns, model.coef_):
    print(f"{feature:15s}: {coef:8.4f}")
print(f"{'Intercept':15s}: {model.intercept_:8.4f}")

### 5.5 視覺化預測結果

In [ ]:
# 繪製實際值 vs 預測值
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5, s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Conversion')
plt.ylabel('Predicted Conversion')
plt.title('Actual vs Predicted Conversion')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("✓ 預測結果視覺化完成")

---
## 6. 檔案儲存與下載示範

In [ ]:
# 將數據儲存為 CSV 檔案
output_filename = '/content/reactor_data.csv'
data.to_csv(output_filename, index=False)
print(f"✓ 數據已儲存至: {output_filename}")

# 查看檔案
!ls -lh /content/*.csv

# 如果要下載到本地電腦，可以執行以下程式碼：
# from google.colab import files
# files.download(output_filename)

In [ ]:
# 如果已掛載 Google Drive，可以將檔案儲存到 Drive
# 這樣檔案就不會在執行階段結束後消失

try:
    # 檢查 Drive 是否已掛載
    import os
    if os.path.exists('/content/drive'):
        drive_path = '/content/drive/My Drive/Colab_Outputs'
        os.makedirs(drive_path, exist_ok=True)
        
        output_path = os.path.join(drive_path, 'reactor_data.csv')
        data.to_csv(output_path, index=False)
        print(f"✓ 數據已儲存至 Google Drive: {output_path}")
    else:
        print("⚠️ Google Drive 未掛載，請先執行 Drive 掛載 cell")
except Exception as e:
    print(f"儲存失敗: {e}")

---
## 7. 進階技巧示範

### 7.1 使用 Magic Commands

In [ ]:
# 測量程式碼執行時間
print("=== Timing Example ===")

# 使用 %timeit 測量單行程式碼
%timeit sum(range(1000))

# 使用 %%time 測量整個 cell
print("\nCell execution time:")

In [ ]:
%%time
# 測量整個 cell 的執行時間
result = sum(range(1000000))
print(f"Sum result: {result}")

In [ ]:
# 查看當前所有變數
print("=== Current Variables ===")
%whos

### 7.2 使用表單功能

Colab 提供互動式表單，方便調整參數而不需修改程式碼。

In [ ]:
#@title Reactor Parameter Settings { run: "auto" }

temperature = 350  #@param {type:"slider", min:300, max:400, step:5}
pressure = 5.0  #@param {type:"number"}
catalyst_type = "Type A"  #@param ["Type A", "Type B", "Type C"]
use_gpu = False  #@param {type:"boolean"}

print("=== Selected Parameters ===")
print(f"Temperature:    {temperature} °C")
print(f"Pressure:       {pressure} bar")
print(f"Catalyst Type:  {catalyst_type}")
print(f"Use GPU:        {use_gpu}")

---
## 8. 總結與課後練習

### 本單元學習重點

✓ 了解 Google Colab 的基本功能與介面操作  
✓ 掌握檔案管理與 Google Drive 掛載方法  
✓ 學會安裝與使用 Python 套件  
✓ 了解如何啟用與檢查 GPU/TPU 資源  
✓ 完成一個完整的化工數據分析流程

### 課後練習

1. **環境熟悉**：嘗試建立一個新的 Colab 筆記本，並練習基本操作
2. **資料處理**：使用自己的數據進行簡單的探索性分析
3. **套件安裝**：安裝並測試其他可能用到的 Python 套件
4. **GPU測試**：如果有啟用 GPU，嘗試比較 CPU 與 GPU 的運算速度差異

### 下一步

完成本單元後，您已經具備使用 Colab 進行 AI 與機器學習開發的基本能力。在後續課程中，我們將深入學習：

- Python 進階資料處理技巧
- 機器學習演算法原理與實作
- 深度學習模型建構
- 化工領域的實際應用案例

**祝您學習順利！**